In [3]:
import torch
import numpy as np
from torch.utils.data import DataLoader

dataset = torch.load("./datasets/dataset_60k_window20k_L4_scaling3_diff.pth")

/tmp/ipykernel_419929/3122187840.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load("./datasets/dataset_60k_window20k_L4_scaling3_diff.pth")


In [4]:
dataset = [(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)) for X, y in dataset]
np.random.shuffle(dataset)

train_dl = DataLoader(dataset[:int(len(dataset)*0.8)], batch_size=32, shuffle=True)
test_dl = DataLoader(dataset[int(len(dataset)*0.8):], batch_size=32, shuffle=True)

In [5]:
torch.cuda.is_available()

True

In [6]:
from aTMi.model import CoalescenceTransformer, num_params
from aTMi.training import train_model, test_model

/home/ubuntu/miniconda/envs/aTMi/lib/python3.9/site-packages/x_transformers/x_transformers.py:507: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/ubuntu/miniconda/envs/aTMi/lib/python3.9/site-packages/x_transformers/x_transformers.py:531: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)


In [7]:
import torch.nn as nn
from x_transformers import Encoder

class CoalescenceTransformer(nn.Module):
    def __init__(self, seq_len, enc_dim, enc_depth,
                 enc_heads, in_dim, out_dim, sum_encoder_dim=True):
        super().__init__()
        self.sum_encoder_dim = sum_encoder_dim
        self.inp = nn.Linear(in_dim, enc_dim)
        self.encoder = Encoder(
            dim=enc_dim, depth=enc_depth,
            heads=enc_heads,ff_glu=True,
            residual_attn=True,
            rotary_pos_emb=True)
        self.out = nn.Linear(enc_dim, out_dim)
    def forward(self, x):
        x = self.inp(x)
        x = self.encoder(x)
        if self.sum_encoder_dim: x = x.sum(dim=1)
        return self.out(x)

In [8]:
few_params = {
    'seq_len': None,
    'enc_dim': 512,
    'enc_depth': 12, 
    'enc_heads': 10, 
    'in_dim': 42,
    'out_dim': 42
}

model = CoalescenceTransformer(**few_params)
print(f'Num. model params.: {num_params(model)}.')
device = torch.device("cuda")
model = model.to(device)

Num. model params.: 53589034.


In [9]:
import torch.nn as nn
import torch.optim as optim
from accelerate import Accelerator


accelerator = Accelerator(mixed_precision='bf16', gradient_accumulation_steps=12) 
criterion = nn.MSELoss()

/home/ubuntu/miniconda/envs/aTMi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [10]:
import logging

def setup_logging(file:str):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    file_handler = logging.FileHandler(file)
    file_handler.setLevel(logging.INFO)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(message)s')
    file_handler.setFormatter(formatter)
    stream_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.addHandler(stream_handler)


setup_logging(file='./logs/training_dataset_60k_window20k_L4_scaling3_diff.log')

In [11]:
learning_rate = 1e-5
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

model, optimizer, train_dl, test_dl = accelerator.prepare(
    model, optimizer, train_dl, test_dl
)

In [12]:
for i in range(0, 10):    
    print(i)
    train_model(model, train_dl, criterion, optimizer, device=device, accelerator=accelerator)
    test_model(model, test_dl, criterion, device=device)    

0


Epoch 1/1: 100%|█████████████████████████████████████| 1500/1500 [01:35<00:00, 15.76batch/s, loss=0.3837]
train,1,1.8094


Epoch [1/1], Avg. Loss: 1.8094


Epoch 1/1: 100%|███████████████████████████████████████| 375/375 [00:07<00:00, 50.45batch/s, loss=0.3880]
test,1,0.4091


Epoch [1/1], Avg. Loss: 0.4091
1


...

Epoch 1/1: 100%|█████████████████████████████████████| 1500/1500 [01:31<00:00, 16.45batch/s, loss=0.3624]
train,1,0.3305


Epoch [1/1], Avg. Loss: 0.3305


Epoch 1/1: 100%|███████████████████████████████████████| 375/375 [00:07<00:00, 50.11batch/s, loss=0.2461]
test,1,0.3067


Epoch [1/1], Avg. Loss: 0.3067
9


Epoch 1/1:  57%|█████████████████████▊                | 861/1500 [00:47<00:38, 16.47batch/s, loss=0.2936]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [13]:
for i in range(10, 25):    
    print(i)
    train_model(model, train_dl, criterion, optimizer, device=device, accelerator=accelerator)
    test_model(model, test_dl, criterion, device=device)    

10


Epoch 1/1: 100%|█████████████████████████████████████| 1500/1500 [01:32<00:00, 16.13batch/s, loss=0.3039]
train,1,0.3171


Epoch [1/1], Avg. Loss: 0.3171


Epoch 1/1: 100%|███████████████████████████████████████| 375/375 [00:07<00:00, 50.12batch/s, loss=0.2441]
test,1,0.3040


Epoch [1/1], Avg. Loss: 0.3040
11


Epoch 1/1:  17%|██████▌                               | 257/1500 [00:14<01:05, 19.00batch/s, loss=0.3354]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [14]:
for epoch in range(25, 30):    
    print(epoch)
    train_model(model, train_dl, criterion, optimizer, device=device, accelerator=accelerator)
    test_model(model, test_dl, criterion, device=device)    

    unwrapped_model = accelerator.unwrap_model(model)
    accelerator.save({
        "model": unwrapped_model.state_dict(),
        "optimizer": optimizer.optimizer.state_dict() 

    }, "./models/model-dataset_60k_window20k_L4_scaling3_diff_checkpoint_" + str(epoch+1) + ".pth")

25


Epoch 1/1: 100%|█████████████████████████████████████| 1500/1500 [01:30<00:00, 16.58batch/s, loss=0.2799]
train,1,0.2759


Epoch [1/1], Avg. Loss: 0.2759


Epoch 1/1: 100%|███████████████████████████████████████| 375/375 [00:07<00:00, 50.45batch/s, loss=0.2652]
test,1,0.2765


Epoch [1/1], Avg. Loss: 0.2765
26


Epoch 1/1:  18%|██████▊                               | 270/1500 [00:14<00:56, 21.58batch/s, loss=0.3017]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [15]:
for epoch in range(30, 50):    
    print(epoch)
    train_model(model, train_dl, criterion, optimizer, device=device, accelerator=accelerator)
    test_model(model, test_dl, criterion, device=device)    

    unwrapped_model = accelerator.unwrap_model(model)
    accelerator.save({
        "model": unwrapped_model.state_dict(),
        "optimizer": optimizer.optimizer.state_dict() 

    }, "./models/model-dataset_60k_window20k_L4_scaling3_diff_checkpoint_" + str(epoch+1) + ".pth")

30


Epoch 1/1: 100%|█████████████████████████████████████| 1500/1500 [01:30<00:00, 16.58batch/s, loss=0.2375]
train,1,0.2691


Epoch [1/1], Avg. Loss: 0.2691


Epoch 1/1: 100%|███████████████████████████████████████| 375/375 [00:07<00:00, 50.52batch/s, loss=0.2406]
test,1,0.2742


Epoch [1/1], Avg. Loss: 0.2742
31


...

Epoch [1/1], Avg. Loss: 0.2643
49


Epoch 1/1: 100%|█████████████████████████████████████| 1500/1500 [01:37<00:00, 15.40batch/s, loss=0.2610]
train,1,0.2445


Epoch [1/1], Avg. Loss: 0.2445


Epoch 1/1: 100%|███████████████████████████████████████| 375/375 [00:07<00:00, 47.94batch/s, loss=0.2867]
test,1,0.2650


Epoch [1/1], Avg. Loss: 0.2650
